# Исследование надежности заемщиков

## Содержание <a id='content'></a>

1. [Загрузка и просмотр данных](#viewing)
2. [Предобработка данных](#processing)
 - [Обработка пропущенных значений](#isna)
 - [Обработка типов данных](#dtypes)
 - [Обработка дубликатов](#duplicates)
 - [Лемматизация целей кредита](#lemmas)
 - [Категоризация данных](#categories)
3. [Исследовательский анализ](#eda)
4. [Общий вывод](#conclusion)

## 1. Загрузка и просмотр данных <a id='viewing'></a>

In [1]:
# импортируем необходимые библиотеки
import pandas as pd
from pymystem3 import Mystem
from collections import Counter # вызов контейнера Counter для подсчета встречающихся лемм
import warnings

In [2]:
warnings.filterwarnings("ignore")

In [3]:
client_stat = pd.read_csv('/datasets/data.csv')

In [4]:
# получим первые 10 строк таблицы
client_stat.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [5]:
# напечатаем общую информацию о таблице
client_stat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Вывод

При первом просмотре данных можно заметить следующие недоработки:
1. столбец days_employed имеет отрицательные значения и неудобный тип данных;
2. столбец education не приведен к одному регистру;
3. столбец total_income лучше перевести в целочисленный тип;
5. имеются дубликаты в столбце purpose;
4. имеются пропуски в столбцах days_employed и total_income.

## 2. Предобработка данных <a id='processing'></a>

### Обработка пропущенных значений <a id='isna'></a>
[Назад к оглавлению](#content)

Скорее всего пропуски в столбцах находятся на одних и тех же строках. Избавимся от пустых значений в столбце total_income:

In [6]:
# найдем строки с пропусками вызовом метода isna() и выведем первые 5
client_stat[client_stat['total_income'].isna()].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


In [7]:
# подсчитаем количество пропусков в столбце вызовом метода sum()
client_stat['total_income'].isna().sum()

2174

В столбце пропущенно 10% значений и мы не можем просто удалить строки с пропусками, так как это повлияет на конечный результат.
Пропуски являются количественными, их мы можем заменить средним значением или медианой. Для показателя уровня дохода медиана является более объективной, так как могут быть большие выбросы.  
Однако для большей верности сначала необходимо поделить клиентов по определенному параметру и заполнять пропуски медианным значением, характерным для каждой такой группы. Группировка клиентов будет происходить по типу занятости, поскольку этот показатель больше всего влияет на получаемый доход.

In [8]:
# сгруппируем таблицу по типу занятости и выведем значения медиан
client_stat.groupby('income_type')['total_income'].median()

income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        150447.935283
компаньон          172357.950966
пенсионер          118514.486412
предприниматель    499163.144947
сотрудник          142594.396847
студент             98201.625314
Name: total_income, dtype: float64

In [9]:
# сохраним сгруппированную по типу занятости таблицу в новую переменную
income_type_grouped = client_stat.groupby('income_type')

In [10]:
# при помощи метода apply() и лямбда-функции заполним пропущенные значения медианами методом fillna()
client_stat['total_income'] = (income_type_grouped['total_income']
                               .apply(lambda row: row.fillna(row.median())))       

In [11]:
# проверяем все ли значения заполнены
client_stat['total_income'].isna().sum()

0

Так как данные столбца days_employed не влияют на решение задачи, отсутствующие значения в нём мы можем обработать медианой без предварительных группировок.

In [12]:
median_days = client_stat['days_employed'].median()
client_stat['days_employed'] = client_stat['days_employed'].fillna(median_days)

### Вывод

Пропуски были выявлены в столбцах с трудовым стажем и ежемесячным доходом. Скорее всего они появились из-за непредоставления клиентом соответствующих данных с места работы, либо справки из пенсионного фонда, если это пенсионеры.  
Заполнение данных медианами, взятыми для каждого источника дохода по отдельности, позволило наиболее правдоподобно восполнить пропущенные значения.

### Обработка типов данных <a id='dtypes'></a>
[Назад к оглавлению](#content)

In [13]:
# переведем значения столбца total_income в целочисленные
# так как изначально тип данных здесь float64, мы можем сразу перевести их методом astype()
client_stat['total_income'] = client_stat['total_income'].astype('int')

In [14]:
# для удобства так же переведем значения столбца days_employed
client_stat['days_employed'] = client_stat['days_employed'].astype('int')

In [15]:
client_stat.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,-5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,-926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,-2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,-152,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,-6929,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,-2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


Посмотрим, какому типу в столбце income_type соответствует положительное и отрицательное количество дней трудового стажа

In [16]:
client_stat.groupby('income_type')['days_employed'].sum()

income_type
безработный            732826
в декрете               -3296
госслужащий          -4636855
компаньон           -10273317
пенсионер          1256208458
предприниматель         -1723
сотрудник           -24621848
студент                  -578
Name: days_employed, dtype: int64

### Вывод

В численных столбцах не встречаются строковые значения, а значит не требуется переводить их в числа, что значительно облегчает преобразование таблицы.  
Значения трудового стажа являются положительными для безработных и пенсионеров, а значит указывают на отсутсвие занятости на данный момент.

### Обработка дубликатов <a id='duplicates'></a>
[Назад к оглавлению](#content)

In [17]:
# применим метод duplicated() и подсчитаем сумму дубликатов
client_stat.duplicated().sum()

54

In [18]:
# выведем список уникальных значений в столбце education
# для лучшей читаемости сделаем это методом value_counts()
client_stat['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

В списке встречаются дубликаты, возникшие из-за записи в разных регистрах.

In [19]:
# приведем данные этого столбца к одному регистру методом str.lower() 
client_stat['education'] = client_stat['education'].str.lower()

In [20]:
# проверяем
client_stat['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

In [21]:
# так же проверим остальные строковые столбцы
client_stat['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
в декрете              1
студент                1
Name: income_type, dtype: int64

In [22]:
client_stat['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

In [23]:
# здесь выделяется значение "Не женат / не замужем", его тоже приведем к нижнему регистру
client_stat['family_status'] = client_stat['family_status'].str.lower()

In [24]:
# проверяем
client_stat['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

In [25]:
client_stat['purpose'].value_counts()

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
операции с жильем                         653
покупка жилья для сдачи                   653
операции с коммерческой недвижимостью     651
жилье                                     647
покупка жилья                             647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
строительство недвижимости                620
покупка своего жилья                      620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

В плане регистра в столбце целей все ОК.

In [26]:
# снова подсчитаем количество дублирующихся строк в таблице
client_stat.duplicated().sum()

71

In [27]:
# и удалим их с обновлением индексов
client_stat = client_stat.drop_duplicates().reset_index(drop=True)

In [28]:
client_stat.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


### Вывод

Дубликатов в исходных данных было менее одного процента, причем их количество выросло после преведения строковых столбцов к одному регистру. Возможно, они возникли из-за повторных заявок, которые были поданы заёмщиками.

### Лемматизация целей кредита <a id='lemmas'></a>
[Назад к оглавлению](#content)

Далее нам нужно выделить основные цели кредита из столбца purposes  с помощью лемматизациии. Для этого напишем функцию, которая будет принимать в качестве аргумента нужную колонку, лемматизировать ее и среди лемм искать соответствие в заранее подготовленном списке категорий, основанном на наиболее часто встречающихся целей кредита. При нахождении такого соответствия функция будет возвращать название этой категории.  
"Жилье" и "недвижимость" объединим в одну категорию.

In [29]:
m = Mystem()

In [30]:
# выведем подсчет лемм
Counter(m.lemmatize(' '.join(client_stat['purpose'])))

Counter({'покупка': 5897,
         ' ': 55023,
         'жилье': 4460,
         'приобретение': 461,
         'автомобиль': 4306,
         'дополнительный': 906,
         'образование': 4013,
         'сыграть': 765,
         'свадьба': 2324,
         'операция': 2604,
         'с': 2918,
         'на': 2222,
         'проведение': 768,
         'для': 1289,
         'семья': 638,
         'недвижимость': 6351,
         'коммерческий': 1311,
         'жилой': 1230,
         'строительство': 1878,
         'собственный': 635,
         'подержать': 853,
         'свой': 2230,
         'со': 627,
         'заниматься': 904,
         'сделка': 941,
         'получение': 1314,
         'высокий': 1374,
         'подержанный': 111,
         'профильный': 436,
         'сдача': 651,
         'ремонт': 607,
         '\n': 1})

После анализа количества встречающихся лемм можно выделить 4 основных категории кредита: недвижимость, автомобиль, образование и свадьба. Цель "жилье" будем относить к недвижимости.  
Необходимо написать функцию, которая будет возвращать одну из данных категорий по значению столбца 'purpose'.

In [31]:
# сохраняем в переменную список категорий
main_purposes = ['недвижимость','автомобиль', 'образование', 'свадьба'] 

def purposes(column):
    lemmas = m.lemmatize(column) #лемматизируем строку таблицы
    for lemma in lemmas:
        if lemma in main_purposes:
            return lemma
        if lemma == 'жилье':
            # если будет встречаться "жилье", функция вернет значение "недвижимость"
            return main_purposes[0] 


In [32]:
# создаем новый столбец и добавляем в него результат работы функции с помощью метода apply()
client_stat['purpose_category'] = client_stat['purpose'].apply(purposes) 

In [33]:
# и печатаем обновленную таблицу
client_stat.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба


In [34]:
# отсортируем количество кредитов по каждой категории
client_stat['purpose_category'].value_counts()

недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: purpose_category, dtype: int64

### Вывод

Из всех целей кредита мы можем выделить 4 основных: недвижимость, автомобиль, образование и свадьба. Наибольшее число заёмов было выдано на приобретение недвижимости, причем их количество превосходит все остальные цели вместе.
Выявить категории кредита помог подсчет наиболее часто встречающихся лемм. Без такого преобразования не удалось бы объективно оценить заёмщиков по данному параметру.

### Категоризация данных <a id='categories'></a>
[Назад к оглавлению](#content)

Выделим словари для более удобного поиска количества должников в столбцах образования и семейного положения. 

In [35]:
# создаем словарь для образования: выделяем education с соответствующим его значению id
edu_dict = client_stat[['education', 'education_id']]
edu_dict

,education,education_id
0,высшее,0
1,среднее,1
2,среднее,1
3,среднее,1
4,среднее,1
...,...,...
21449,среднее,1
21450,среднее,1
21451,среднее,1
21452,среднее,1


In [36]:
# удалим из него все дубликаты
edu_dict = edu_dict.drop_duplicates().reset_index(drop=True)
edu_dict

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


In [37]:
# сделаем такой же словарь для столбца семейного положения
family_dict = (client_stat[['family_status', 'family_status_id']]
               .drop_duplicates()
               .reset_index(drop=True))
family_dict

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,не женат / не замужем,4


Для проведения категоризации по уровню дохода разобьем этот столбец на квартели и на их основании сгруппируем данные:

In [38]:
pd.qcut(client_stat['total_income'], 4) # выведем квартели в столбце total_income методом qcut()

0        (195820.25, 2265604.0]
1          (107623.0, 142594.0]
2         (142594.0, 195820.25]
3        (195820.25, 2265604.0]
4         (142594.0, 195820.25]
                  ...          
21449    (195820.25, 2265604.0]
21450     (142594.0, 195820.25]
21451     (20666.999, 107623.0]
21452    (195820.25, 2265604.0]
21453     (20666.999, 107623.0]
Name: total_income, Length: 21454, dtype: category
Categories (4, interval[float64]): [(20666.999, 107623.0] < (107623.0, 142594.0] < (142594.0, 195820.25] < (195820.25, 2265604.0]]

Получились следующие 4 группы доходов клиентов:  
 <= 107623  
 <= 142594  
 <= 195820.25  
 <= 2265604  

In [39]:
# создаем функцию для группировки по уровню дохода
def income_group(income):
    if income <= 107623:
        return 'до 107623'
    if income <= 142594:
        return 'от 107625 до 142594'
    if income <= 195820.25:
        return 'от 142595 до 195820,25'
    return 'от 195821'

# создаем новый столбец в таблице на основе возвращенных результатов функции
client_stat['income_group'] = client_stat['total_income'].apply(income_group)

In [40]:
# протестируем эту функцию
income_group(50000)

'до 107623'

In [41]:
income_group(142594)

'от 107625 до 142594'

In [42]:
income_group(142595)

'от 142595 до 195820,25'

In [43]:
income_group(9999999)

'от 195821'

In [44]:
client_stat.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category,income_group
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,от 195821
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,от 107625 до 142594
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,"от 142595 до 195820,25"
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,от 195821
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,"от 142595 до 195820,25"


In [45]:
# подсчитаем количество кредитов по группам дохода
client_stat['income_group'].value_counts()

от 107625 до 142594       5479
от 195821                 5364
до 107623                 5364
от 142595 до 195820,25    5247
Name: income_group, dtype: int64

### Вывод

Выделение словарей для столбцов с семейным положением и образованием клиентов заметно упрощает дальнейшую обработку таблицы, так как снижается вероятность ошибки при вводе данных.  
Разбиение на квантили помогло определить основные равные группы клиентов по уровню получаемого дохода, что позволит наиболее рационально выявить процент должников по этому параметру.

## 3. Исследовательский анализ <a id='eda'></a>
[Назад к оглавлению](#content)

### Есть ли зависимость между наличием детей и возвратом кредита в срок?

Сформируем свобную таблицу, в которой сгруппируем клиентов по количеству детей и посмотрим сколько просрочек было в каждой группе

In [46]:
# выделим общее количество клиентов в группе, число должников и их долю
client_stat.pivot_table(index='children', 
                        values='debt', 
                        aggfunc=['count', 'sum', 'mean'])

,count,sum,mean
,debt,debt,debt
children,,,
-1,47,1,0.021277
0,14091,1063,0.075438
1,4808,444,0.092346
2,2052,194,0.094542
3,330,27,0.081818
4,41,4,0.097561
5,9,0,0.000000
20,76,8,0.105263


В таблице стало видно артефактное значение -1. Можно предположить, что так обозначают детей, которые с момента выдачи кредита стали совершеннолетними.  
Для удобства можно категоризировать количество детей по аналогии с уровнем дохода. она будет делить клиентов на 3 категории - бездетные, 1-2 ребенка и многодетные.

In [47]:
def children_categoty(children):
    if children < 1:
        return 'бездетные'
    if children < 3:
        return '1-2 ребенка'
    return 'многодетные'

In [48]:
client_stat['children_categoty'] = client_stat['children'].apply(children_categoty)

In [49]:
# переделаем сводную таблицу с учетом категоризации
client_stat.pivot_table(index='children_categoty', 
                        values='debt', 
                        aggfunc=['count', 'sum', 'mean'])

,count,sum,mean
,debt,debt,debt
children_categoty,,,
1-2 ребенка,6860,638,0.093003
бездетные,14138,1064,0.075258
многодетные,456,39,0.085526


### Вывод

Первый ответ на поставленный вопрос - наибольшее количество просрочек по кредиту имеют клиенты, у которых 1-2 ребенка в семье, их даже больше, чем у многодетных. Наиболее высокий процент добросовестных плательщиков среди тех, у кого дети отсутствутют.

### Есть ли зависимость между семейным положением и возвратом кредита в срок?

Посмотрим на те же показатели для клиентов с разным семейным положением:

In [50]:
client_stat.pivot_table(index='family_status', 
                        values='debt', 
                        aggfunc=['count', 'sum', 'mean'])

,count,sum,mean
,debt,debt,debt
family_status,,,
в разводе,1195,85,0.071130
вдовец / вдова,959,63,0.065693
гражданский брак,4151,388,0.093471
женат / замужем,12339,931,0.075452
не женат / не замужем,2810,274,0.097509


### Вывод

Больше всего просрочек по кредиту имеют неженатые и незамужние клиенты, а также состоящие в гражданском браке - на них приходится почти 10 процентов. Самый маленький процент среди заемщиков, находящихся в статусе вдовца/вдовы. 
В целом наблюдается картина, что зарегестрированные в браке клиенты являются более добросовестными.

### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Сводная таблица для разного уровня дохода будет выглядеть следующим образом:

In [51]:
client_stat.pivot_table(index='income_group', 
                        values='debt', 
                        aggfunc=['count', 'sum', 'mean'])

,count,sum,mean
,debt,debt,debt
income_group,,,
до 107623,5364,427,0.079605
от 107625 до 142594,5479,483,0.088155
"от 142595 до 195820,25",5247,448,0.085382
от 195821,5364,383,0.071402


### Вывод

Почти одинаковый процент просрочек (около 9) имеют клиенты, относящиеся к двум средним группам с доходом приблизительно от 100 до 200 тыс. Клиенты с более высоким заработком являются самыми добросовестными - процент их просрочек равен 7-и. Примерно 8% задолженностей среди клиентов с доходом до 107 тыс. 

### Как разные цели кредита влияют на его возврат в срок?

Заключительная сводная таблица сгруппирует клиентов по целям кредита:

In [52]:
client_stat.pivot_table(index='purpose_category', 
                        values='debt', 
                        aggfunc=['count', 'sum', 'mean'])

,count,sum,mean
,debt,debt,debt
purpose_category,,,
автомобиль,4306,403,0.093590
недвижимость,10811,782,0.072334
образование,4013,370,0.092200
свадьба,2324,186,0.080034


### Вывод

Самая популярная цель кредита одновременно является самой возвращаемой. Больше всего должников среди тех, кто брал кредит на автомобиль, чуть меньше - на образование.

## 4. Общий вывод <a id='conclusion'></a>
[Назад к оглавлению](#content)

В этом проекте был представлен анализ данных клиентов, предоставленных кредитным отделом банка, по влиянию различный параметров на вероятность просрочек по кредиту. Было выделено 4 основных влияющих параметра: количество детей, уровень дохода, ежемесячный доход и цель кредита.  
В исходной таблице, объёмом 21525 строк, выявилось несколько недоработок, из-за которых дальнейший анализ был бы невозможен: а именно пропущенные значения  в столбцах с количеством отработанных дней и уровнем дохода, дубликаты, неудобный тип данных в некоторых численных значениях, запись данных в разном регистре. После обработки таблицы количество ее строк сократилось на 71 за счет удаления дубликатов, пропущенные значения были заменены сгруппированными медианами, а значения с плавающей точкой заменены на целочисленные.     
В ходе работы для более удобного анализа в таблицу было добавлено несколько дополнительных столбцов:
1. Категория цели кредита. В нем при помощи лемматизации было выделено 4 основные цели, на которые клиенты банка брали заём: недвижимость, автомобиль, образование и свадьба. Выявление этих целей происходило путём подсчета наиболее часто встречающихся лемм.
2. Группа дохода. Здесь клиенты были разделены на 4 категории по получаемому ежемесячному доходу: до 107623, до 142594, до  195820.25 и выше, при этом размах значений был от 20666 до 2265604.  

По результатам проделанной работы можно сказать, что все выдвинутые гипотезы подтвердились: на возврат кредита в срок влияют много факторов, среди которых наличие детей, семейное положение, уровень дохода и цели заёма. Кредитному отделу банка нужно обязательно учитывать эти показатели для принятия решения по каждому клиенту.  
Отсутствие детей у заёмщика положительным образом отражается на его платежеспособности - только 7,5% просрочек. Клиенты банка с 1-2 имеют долю больше 9%, при этом многодетные дети находятся посередине - 8,5 %
Практически 10% задолженностей, что является большинством, имеют неженатые и незамужние клиенты, а также состоящие в гражданском браке. Вдовцы и вдовы имеют меньше всего просрочек.  
Самыми добросовестными плательщиками являются клиенты с доходом приблизительно от 200 тыс. - доля просрочек у них около 7%. Примерно на 1,5-2% больше задолженностей у заёмщиков с доходом от 100 до 200.  
Охотнее всего возвращают кредит на недвижимость, который одновременно является самым часто берущимся - из почти 11000 выданных кредитов около 7% оказались с просрочками. В автомобильных кредитах был выявлен наибольший показатель задолженности - 9,3%.  
В целом доля должников не привышает 10% по каждому показателю, что примерно соответсвует общей доли должников в исходных данных  - около 8 %. 